In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras as kr

2024-11-26 09:53:40.570360: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mnist = kr.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
kr.utils.normalize(x_train,axis=1)
kr.utils.normalize(x_test,axis=1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

train_data = {'X':x_train, 'Y':y_train}
test_data = {'X':x_test, 'Y':y_test}

In [3]:
model = kr.models.Sequential([
  kr.layers.Flatten(),
  kr.layers.Dense(128, activation='relu'),
  kr.layers.Dense(128, activation='relu'),
  kr.layers.Dense(10, activation='softmax')
])


In [4]:
class PopulationBasedOptimizer:

    def __init__(self, model, NumberOfAgents, CutOffMeasure, NumberOfLearningIterations, InitialHyperParameter):
        self.listOfAgents = [kr.models.clone_model(model) for _ in range(NumberOfAgents)]
        self.HyperParameters = InitialHyperParameter
        self.NumberOfAgents = NumberOfAgents
        self.InitialHyperParameter = InitialHyperParameter
        self.NumberOfLearningIterations = NumberOfLearningIterations
        self.CutOffMeasure = CutOffMeasure
        self.Iterations = NumberOfLearningIterations
        for Agent, rate in zip(self.listOfAgents,self.HyperParameters):
            optimizer = kr.optimizers.Adam(learning_rate=rate, clipnorm=10)
            Agent.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    def PBTOptimise(self, train_data, test_data):
        for iter in range(self.NumberOfLearningIterations):
            for model in self.listOfAgents:
                model.fit(train_data.get('X'), train_data.get('Y'), epochs=1, batch_size=10)
            current_loss = np.zeros(self.NumberOfAgents)
            for i,model in enumerate(self.listOfAgents):
                loss, accuracy = model.evaluate(test_data.get('X'), test_data.get('Y'), verbose=1)
                current_loss[i] = loss
            Best = np.argmin(current_loss)
            BadIndices = np.argsort(current_loss)[int(self.NumberOfAgents * self.CutOffMeasure):]
            for index in BadIndices:
                # Best_weights = self.listOfAgents[Best].get_weights()
                # self.listOfAgents[index].set_weights(self.listOfAgents)[Best].get_weights() + np.random.normal(0,1,Best_weights.shape))
                noisy_weights = [
                    weight + tf.random.normal(shape=weight.shape, mean=0.0, stddev=0.1).numpy()
                    for weight in self.listOfAgents[Best].get_weights()
                ]
                self.listOfAgents[index].set_weights(noisy_weights)
                optimizer = kr.optimizers.Adam(learning_rate=self.InitialHyperParameter[Best], clipnorm=10)
                self.InitialHyperParameter[index] = self.InitialHyperParameter[Best]
                self.listOfAgents[index].compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        for model in self.listOfAgents:
            model.fit(train_data.get('X'), train_data.get('Y'), epochs=1)
        current_loss = np.zeros(self.NumberOfAgents)
        for i,model in enumerate(self.listOfAgents):
            current_loss[i], accuracy = model.evaluate(test_data.get('X'), test_data.get('Y'), verbose=1)
            BadIndices = np.argsort(current_loss)[int(self.NumberOfAgents * self.CutOffMeasure):]
        Best = np.argmin(current_loss)
        # Please help
        return self.listOfAgents[Best]

In [ ]:
InitialHyperParameter = np.random.normal(0,1,10)
PBT = PopulationBasedOptimizer(model=model, NumberOfAgents=10, CutOffMeasure=0.4, NumberOfLearningIterations=10, InitialHyperParameter=InitialHyperParameter)
best_model = PBT.PBTOptimise(train_data = train_data, test_data = test_data)

6000/6000 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.0917 - loss: 2921575535869952.0000
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.0992 - loss: 1738089378611200.0000
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.1006 - loss: 149696135823360.0000
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.1034 - loss: 104867544996773888.0000
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.0917 - loss: 23982300664430592.0000
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.1099 - loss: 2.7826
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.1015 - loss: 3.4979
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.0981 - loss: 81613337925779456.0000
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.1060 - loss: 35.1237
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.1045 - loss: 317090657468416.0000
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0911 - loss: 44600044208259072.0000
313/313 ━━━━━━━━━━━━━

In [ ]:
best_model.evaluate(test_data.get('X'),  test_data.get('Y'), verbose=5)

In [ ]:

model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["accuracy"])
model.fit(train_data.get('X'), train_data.get('Y'), epochs=10)
model.evaluate(test_data.get('X'),  test_data.get('Y'), verbose=2)